# 📦 Section 1 – Chargement et nettoyage des données

Nous chargeons ici **toutes les sources** de données pour un site :  
- NOAA (2 stations)
- Meteostat (2 stations)
- ERA5
- OpenMeteo
- NASA POWER

Toutes les vitesses sont **en m/s**. Aucune valeur aberrante n’est supprimée.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import weibull_min, genextreme

# Chargement des fichiers (à adapter selon le site analysé)
sources = {
    "noaa1": "noaa_station1_PIOLENC.csv",
    "noaa2": "noaa_station2_PIOLENC.csv",
    "meteostat1": "meteostat1_PIOLENC.csv",
    "meteostat2": "meteostat2_PIOLENC.csv",
    "era5": "era5_daily_PIOLENC.csv",
    "nasa": "nasa_power_PIOLENC.csv",
    "openmeteo": "openmeteo_PIOLENC.csv"
}

dfs = {}
for key, path in sources.items():
    df = pd.read_csv(path, parse_dates=["time"])
    dfs[key] = df


# 📊 Section 2 – Présentation des données récupérées

Visualisation des **vents moyens** issus de toutes les sources.  
Deux lignes horizontales seront ajoutées plus tard :
- 📘 Building Code – vent moyen
- ⚡ Building Code – vent en rafale


In [ ]:
plt.figure(figsize=(14, 7))
for name, df in dfs.items():
    if "windspeed_mean" in df.columns:
        plt.plot(df["time"], df["windspeed_mean"], label=name)

plt.axhline(y=999, color="gray", linestyle="--", label="BC – Vent moyen (à insérer)")
plt.title("Vent moyen – comparaison entre sources")
plt.xlabel("Date")
plt.ylabel("Vent moyen (m/s)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


# 🌀 Section 3 – Analyse des vents moyens

Nous ajustons ici la **loi de Weibull** sur chaque source de vent moyen.  
Et nous en déduisons la **valeur de retour 50 ans**.


In [ ]:
T = 50
for name, df in dfs.items():
    if "windspeed_mean" in df.columns:
        data = df["windspeed_mean"].dropna()
        if len(data) > 0:
            params = weibull_min.fit(data, floc=0)
            v_return = weibull_min.ppf(1 - 1/T, *params)
            print(f"{name} → Vent moyen 50 ans : {v_return:.2f} m/s")


# 🌪️ Section 4 – Analyse des vents rafales

On utilise ici les colonnes `gust` ou `windspeed_gust`.  
On ajuste une **loi GEV complète** sur chaque source, et on extrait :
- Le paramètre \( \xi \)
- Le vent de retour 50 ans


In [ ]:
for name, df in dfs.items():
    gust_col = None
    for col in ["gust", "windspeed_gust"]:
        if col in df.columns:
            gust_col = col
            break
    if gust_col:
        data = df[gust_col].dropna()
        if len(data) > 0:
            gev_params = genextreme.fit(data)
            xi = gev_params[0]
            v_return = genextreme.ppf(1 - 1/T, *gev_params)
            print(f"{name} → Rafale 50 ans : {v_return:.2f} m/s (ξ = {xi:.3f})")


# ✅ Section 5 – Bilan récapitulatif

Nous avons :
- Comparé visuellement toutes les sources
- Ajusté Weibull pour les vents moyens
- Ajusté GEV pour les rafales
- Calculé les valeurs de retour 50 ans

🔜 À venir :
- Ajout des valeurs des Building Codes via Excel
- Extension des rafales estimées via les vents moyens
- Export PDF automatique du notebook complet
